In [1]:
%load_ext autoreload
%autoreload 1

In [3]:
from collections import defaultdict
import os, sys
sys.path.insert(0, "/mnt/d/dev/git/miRExplore/python/")


import scispacy
import spacy
from spacy import displacy

nlp = spacy.load('/mnt/f/spacy/en_core_sci_lg-0.2.5/en_core_sci_lg/en_core_sci_lg-0.2.5/')



In [7]:
! ls /mnt/f/spacy/en_ner_bionlp13cg_md-0.2.5/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.5

meta.json  ner	parser	tagger	tokenizer  vocab


In [8]:
nlp_ent = spacy.load("/mnt/f/spacy/en_ner_bionlp13cg_md-0.2.5/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.5")

In [26]:
import textmining.MirGeneRelCheck
from intervaltree import Interval, IntervalTree

In [37]:

def analyse(testCase, dist=80):
    relCheck = textmining.MirGeneRelCheck.MirGeneRelCheck()

    doc = nlp(testCase[0])
    entDoc = nlp_ent(testCase[0])
    
    enttree = IntervalTree()
    for ent in entDoc.ents:
        enttree.addi(ent.start_char, ent.end_char, (ent.label_, ent.text))

    
    lWord = doc[testCase[1]]
    rWord = doc[testCase[2]]

    for t in doc:
        
        possTokenEnts = enttree.overlap(t.idx, t.idx+len(t.text))
        ents = set([x.data for x in possTokenEnts])
        
        print(t.idx, t.idx+len(t.text), t.i, t, t.pos_, t.dep_, t.head, t.head.idx, ents)

    print(lWord, rWord)

    if not "mir" in str(lWord).lower() and not "let" in str(lWord).lower():
        tmp = lWord
        lWord = rWord
        rWord = tmp
        
    print(lWord, rWord)
    
    checkResults = relCheck.checkRelation(doc, lWord, rWord, verbose=True)
    print(checkResults)
    
    displacy.render(doc, style="dep", options={"compact": True, "distance": dist})
    
    entDoc = nlp_ent(testCase[0])
    
    
    displacy.render(entDoc, style="ent", options={"compact": True, "distance": dist})
    
    return enttree

In [38]:
%aimport textmining.MirGeneRelCheck

testcase = (u"Functional association between miR-124a and CDK6 in medulloblastoma was established using luciferase assays.", 4, 6)

enttree = analyse(testcase)

0 10 0 Functional ADJ amod association 11 set()
11 22 1 association NOUN nsubjpass established 72 set()
23 30 2 between ADP case miR-124a 31 set()
31 39 3 miR-124a NOUN nmod association 11 {('GENE_OR_GENE_PRODUCT', 'miR-124a')}
40 43 4 and CCONJ cc miR-124a 31 set()
44 48 5 CDK6 NOUN conj miR-124a 31 {('GENE_OR_GENE_PRODUCT', 'CDK6')}
49 51 6 in ADP case medulloblastoma 52 set()
52 67 7 medulloblastoma NOUN nmod association 11 {('CANCER', 'medulloblastoma')}
68 71 8 was AUX auxpass established 72 set()
72 83 9 established VERB ROOT established 72 set()
84 89 10 using VERB xcomp established 72 set()
90 100 11 luciferase NOUN compound assays 101 {('GENE_OR_GENE_PRODUCT', 'luciferase')}
101 107 12 assays NOUN dobj using 84 set()
107 108 13 . PUNCT punct established 72 set()
and in
in and
Conjuncts [CDK6, miR-124a]
neighbour CDK6 in
neighbour CDK6 in
Conjunction before fc miR-124a
fc add children for miR-124a
Conjuncts [miR-124a, CDK6]
neighbour miR-124a and
neighbour miR-124a and
Conjunct

In [36]:
[x.data for x in enttree.overlap(31,39)]

[('GENE_OR_GENE_PRODUCT', 'miR-124a')]

In [18]:
analyse(
(
    "miR-10a inhibits dendritic cell activation and Th1/Th17 cell immune responses in IBD.",
    0,
    6
)
)

0 0 miR-10a NOUN nsubj inhibits 8
8 1 inhibits VERB ROOT inhibits 8
17 2 dendritic ADJ amod activation 32
27 3 cell NOUN compound activation 32
32 4 activation NOUN dobj inhibits 8
43 5 and CCONJ cc activation 32
47 6 Th1/Th17 NOUN compound responses 68
56 7 cell NOUN compound responses 68
61 8 immune ADJ amod responses 68
68 9 responses NOUN conj activation 32
78 10 in ADP case IBD 81
81 11 IBD NOUN nmod activation 32
84 12 . PUNCT punct inhibits 8
miR-10a Th1/Th17
miR-10a Th1/Th17
Conjuncts [responses, activation]
neighbour responses in
neighbour responses in
Conjunction before fc activation
fc add children for activation
fc add children for activation
Conjuncts [activation, responses]
neighbour activation and
neighbour activation and
Conjunction before fc responses
fc add children for responses
Conjunctions
activation [dendritic, activation, immune, responses, in, IBD]
SDP sent: miR-10a inhibits dendritic cell activation and Th1/Th17 cell immune responses in IBD.
SDP     : [(miR-10a

miR-10a 0 7 GENE_OR_GENE_PRODUCT
dendritic cell 17 31 CELL
Th1/Th17 cell 47 60 CELL
IBD 81 84 TISSUE


In [15]:
analyse(
(
    "miR-10a inhibits dendritic cell activation and Th1/Th17 cell immune responses in IBD.",
    0,
    6
)
)

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'label_'